In [47]:
import numpy as np
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
import matplotlib.pyplot as plt  # Библиотека для визуализации результатов 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler

In [11]:
df = pd.read_csv('/Users/kseniagorchakova/Library/Mobile Documents/com~apple~CloudDocs/Education/!Profession_DS/Машинное обучение/Лекция_2_ Логистическая регрессия и SVM/Материалы к ДЗ/adult.csv',na_values = '?')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [27]:
df['capital-loss'].unique() 

array([   0, 1721, 1876, 2415, 1887,  625, 1977, 2057, 1429, 1590, 1485,
       2051, 2377, 1672, 1628, 1902, 1602, 1741, 2444, 1408, 2001, 2042,
       1740, 1825, 1848, 1719, 3004, 2179, 1573, 2205, 1258, 2339, 1726,
       2258, 1340, 1504, 2559, 1668, 1974, 1980, 1564, 2547, 2002, 1669,
       1617,  323, 3175, 2472, 2174, 1579, 2129, 1510, 1735, 2282, 1870,
       1411, 1911, 1651, 1092, 1762, 2457, 2231, 2238,  653, 1138, 2246,
       2603, 2392, 1944, 1380, 2465, 1421, 3770, 1594,  213, 2149, 2824,
       1844, 2467, 2163, 1816, 1648, 2206, 2352,  419, 1539,  880,  810,
        974, 4356, 2489, 1755, 3683, 2267, 2080,  155, 3900, 2201, 2754])

In [33]:
# В качестве признаков беру образование, возраст, расу и кол-во рабочих часов
selectedColumns = data [['education', 'age','race', 'hours-per-week']]

In [34]:
# Переводим категориальные переменные в числовые
x = pd.get_dummies (selectedColumns, columns = ['education', 'race'])

In [35]:
x.head()

,age,hours-per-week,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,...,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,40,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,50,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,28,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,44,40,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,18,30,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


### Т.к. целевая переменная, которую необходимо предсказать строковая, заменим ее на значения с помощью LabelEncoder (0 и 1)

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [23]:
le.fit( df['income'])

LabelEncoder()

In [25]:
# Записываем в переменную y трасформированное значение
y = pd.Series (data = le.transform(df['income']))

In [18]:
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

### Модель логистической регрессии

In [20]:
model = LogisticRegression()

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [44]:
# обучаем на части датасета (train)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0, 0, ..., 0, 0, 0])

In [46]:
model.score(x_test,y_test)

0.7941035965331331

### SVC

In [48]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [49]:
clf.score(x_test, y_test)

0.8014741008667167

In [50]:
svc = SVC(gamma='auto')
svc.fit(x_train, y_train)

SVC(gamma='auto')

In [51]:
svc.score(x_test, y_test)

0.8007234013512591